In [6]:
import random
from deap import creator, base, tools, algorithms
import numpy as np
import keras

In [7]:
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
def create_individual(type):
    if type == 'ASSETS':
        random_list = np.random.normal(0, 3, 26)
    elif type == 'EQ':
        random_list = np.random.normal(0, 3, 29)
    else:
        raise TypeError

    return list(random_list/sum(random_list))

In [68]:
NN_model = keras.models.load_model('./saved_model.h5')

class IndividualStructure(object):
    ASSETS_LEN = 26
    EQ_AND_LIAB_LEN = 29

    def __init__(self, assets, eq_and_liab):
        self.structure = assets() + eq_and_liab()
        self.predecessors = [assets() + eq_and_liab()]
        self.historic_fitness = []
        self.fitness = IndividualStructure.magic_func(self.predecessors[-1] + self.structure)

    def compete_with(self, potential_successor_structure):
        return IndividualStructure.magic_func(potential_successor_structure + self.structure)

    def set_the_winner(self, desc_struct, desc_fitness):
        if desc_fitness > 0.8 * self.fitness and desc_fitness > 0:
            self.predecessors.append(toolbox.clone(self.structure))
            self.structure = desc_struct
            self.historic_fitness.append(self.fitness)
            self.fitness = desc_fitness

    @staticmethod
    def create_descendant(struct_a, struct_b):
        descendant = [value[random.randint(0,1)] for value in zip(struct_a, struct_b)]
        assets_sum = sum(descendant[:IndividualStructure.ASSETS_LEN])
        eq_and_liab_sum = sum(descendant[IndividualStructure.ASSETS_LEN:])
        descendant[:IndividualStructure.ASSETS_LEN] = list(map(lambda x: x/assets_sum, descendant[:IndividualStructure.ASSETS_LEN]))
        descendant[IndividualStructure.ASSETS_LEN:] = list(map(lambda x: x/eq_and_liab_sum, descendant[IndividualStructure.ASSETS_LEN:]))

        return descendant

    @staticmethod
    def magic_func(data):
        return NN_model.predict([data])[0][0]

In [69]:
# def main():
toolbox = base.Toolbox()
POPULATION_SIZE = 50
GENERATIONS = 30
DESC_PER_PARENTS = 2
toolbox.register("individual", IndividualStructure, lambda: create_individual('ASSETS'), lambda: create_individual('EQ'))
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=POPULATION_SIZE)

pop = toolbox.population()

for gen in range(GENERATIONS):
    print(f'Generation: {gen}')
    randomized_indexes = random.sample(list(range(len(pop))), k=len(pop))
    paired_indexes = list(zip(randomized_indexes[::2], randomized_indexes[1::2]))
    parents_pairs = map(lambda x: (pop[x[0]], pop[x[1]]), paired_indexes)
    for parent_a, parent_b in parents_pairs:
        descendants = [
            IndividualStructure.create_descendant(parent_a.structure, parent_b.structure)
            for _ in range(DESC_PER_PARENTS)
        ]
        parent_a.set_the_winner(descendants[0], parent_a.compete_with(descendants[0]))
        parent_b.set_the_winner(descendants[1], parent_b.compete_with(descendants[1]))

Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
Generation: 10
Generation: 11
Generation: 12
Generation: 13
Generation: 14
Generation: 15
Generation: 16
Generation: 17
Generation: 18
Generation: 19
Generation: 20
Generation: 21
Generation: 22
Generation: 23
Generation: 24
Generation: 25
Generation: 26
Generation: 27
Generation: 28
Generation: 29


In [70]:
max_plus_sum = sorted(pop, key=lambda x: sum(x.historic_fitness) + x.fitness, reverse=True)

In [71]:
longest_growth = sorted(pop, key=lambda x: len(x.historic_fitness), reverse=True)

In [80]:
print(longest_growth[0].historic_fitness, longest_growth[0].fitness)

[0.1595152, 1.6095779, 6.230712, 6.7264137, 8.082814, 8.940534, 13.568923] 12.158284


In [79]:
print(max_plus_sum[0].historic_fitness, max_plus_sum[0].fitness)

[-3.5601437, 20.661863] 1203.7094
